# Project Goal 

Разработаем чатбот-барохольщик. На входе он принимает любой запрос от пользователя, затем классифицирует его на продуктовый или другой тип запроса.
Если запрос продуктовый, то на выходе пользователь получит id товара, который подходит под его запрос, если запрос другой - то бот перенаправит вопрос в болталку и даст ответ на вопрос, который был ближе всего к вопросу, который был взят из корпуса текста с форумов на mail.ru.

В процессе проекта будем использовать векторизацию текста, модель Word2Vec, логистическую регрессию.

In [1]:
import os
import re
import pickle
import string
import annoy

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
nltk.download('stopwords')

from gensim.models import Word2Vec
from joblib import dump, load
from pymorphy2 import MorphAnalyzer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation

import numpy as np
import tqdm
from tqdm import tqdm_notebook
import pandas as pd

[nltk_data] Downloading package stopwords to /home/nlp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1. Classifier
Обучим классификатор - будем разделять вопросы от пользователя на продуктовые запросы и все остальные.

Предобработаем ответы mail.ru из файла: к каждому вопросу присоединим 1 ответ и запишем в файл на будущее. Это позволит нам сэкономить время и ресурсы при дальнейшем препроцессинге текста.

In [2]:
question = None
written = False

'''Идем по всем записям, берем первую строку как вопрос и после знака --- находим ответ'''
with open('prepared_answers.txt', 'w') as fout:
    with open('answers.txt', 'r') as fin:
        for line in tqdm_notebook(fin):
            if line.startswith('---'):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace('\t', ' ').strip() + '\t' + line.replace('\t', ' '))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

/tmp/ipykernel_6758/1279344030.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [3]:
'''Функция для нормализации текста - приведение к нижнему регистру, удаление стоп-слов, знаков пунктуации, латинских букв, лемматизация'''
def preprocess_text(txt: str, sw = []) -> str:
    exclude = list(punctuation)
    '''Стоп-слова из библиотеки nltk'''
    sw_1 = list(stopwords.words('russian'))
    morpher = MorphAnalyzer()
    txt = str(txt)
    '''Делим предложение на слова по знакам препинания, приводим все слова к нижнему регистру, удаляем знаки препинания, стоп-слова и цифры'''
    tokenizer = RegexpTokenizer("\w+|[^\w\s]+")
    txt = [i.lower() for i in tokenizer.tokenize(txt) if i not in exclude and i not in sw and i not in sw_1 and len(i) > 2 and \
          re.search('(\d+)|\)|\(|\.|\:|\;|\!|\?|\<|\>|[A-Za-z]', i) is None]
    '''Делаем лемматизацию'''
    txt = [morpher.parse(word)[0].normal_form for word in txt]
    return " ".join(txt)

In [4]:
'''Дополнительные стоп-слова'''
sw = ['всё', 'тому', 'представьте', 'чьё', 'смысле', 'статься', 'знаешь', 'получается', 'или', 'выяснилось', 'удивительно',
      'которых', 'оти', 'значится', 'сказано', 'никогда', 'об', 'образом', 'моему', 'самом', 'свыше', 'во-вторых', 'как', 'чья', 'иметь', 
      'следовательно', 'серьезно', 'точки', 'точнее', 'поди', 'бы', 'но', 'само', 'осле', 'конено', 'около', 'из-за', 'отому', 'скажем',
      'новость', 'достаточно', 'издревле', 'когда', 'любом', 'ваша', 'мол', 'конец', 'словно', 'никто', 'уж', 'да', 'актуальный', 'попросту',
      'бесспорно', 'долженствовать', 'было', 'которая', 'хорошо', 'случайно', 'хоть', 'ко', 'видать', 'повседневной', 'этот', 'позволь',
      'наверно', 'прочему', 'самым', 'буквально', 'здесь' 'воистину', 'право', 'ничего', 'понятно', 'разнообразный', 'вообразите', 'июль',
      'эта', 'тогда', 'дальше', 'конечно', 'наоборот', 'бывает', 'твоя', 'мне', 'полагать', 'важно', 'нам', 'представленный', 'проще',
      'очередной', 'выясняется', 'странно', 'устоявшееся', 'моя', 'всевозможный', 'долженствующий', 'прочий', 'где', 'откровенно', 'принято',
      'беспрецедентный', 'целом', 'один', 'нашей', 'абсолютно', 'следствие', 'также', 'они', 'после', 'согласиться', 'реальный', 'по-твоему',
      'имеется', 'лет', 'небось', 'казалось', 'своему', 'скорее', 'неизгладимый', 'напротив', 'март', 'наверняка', 'обычно', 'нынче',
      'моей', 'примечательно', 'против', 'наконец', 'так', 'тыс', 'какой', 'много', 'ней', 'если', 'должно', 'ваш', 'нее', 'быть', 'этом',
      'чем', 'возможно', 'вестимо', 'знать', 'вроем', 'выражаясь', 'одним', 'теерь', 'кажется', 'многие', 'от', 'есть', 'себе', 'просто',
      'всяком', 'даже', 'ничто', 'обычаю', 'знает', 'исходя', 'несколько', 'предельно', 'ли', 'она', 'ерез', 'вас', 'ведь', 'имеются', 'более',
      'он', 'кто', 'по', 'уть', 'свою' 'твоему', 'них', 'кстати', 'од', 'крайне', 'ри', 'говори', 'примеру', 'их', 'вашему', 'всякого',
      'исключение', 'через', 'того', 'год', 'оказывается', 'по-ихнему', 'вишь', 'конце', 'вашего', 'тоб', 'кб', 'пожалуйста', 'сейас',
      'вам', 'эй', 'над', 'естественно', 'без', 'нибудь', 'нечего', 'еред', 'куда', 'вообще-то', 'менее', 'по-вашему', 'хм', 'что', 'про',
      'всей', 'им', 'наша', 'кажись', 'твоего', 'своей', 'наверное', 'собой', 'нами', 'предположительно', 'мы', 'всех', 'по-моему', 'исстари',
      'себя', 'некто', 'иногда', 'совершенно', 'для', 'вы', 'этих', 'как-то', 'разумеется', 'ему', 'всегда', 'чего', 'другие', 'во-первых',
      'позвольте', 'вип', 'очевидно', 'так-то', 'замыслу', 'нарочно', 'короче', 'то', 'помилуйте', 'сути', 'по-хорошему', 'иначе', 'нередко',
      'чтобы', 'всего', 'все', 'надо', 'весь', 'раз', 'известно', 'из', 'свой', 'по-видимому', 'тут', 'под', 'мой', 'ул', 'луше', 'ей', 'только',
      'вероятно', 'видимо', 'руб', 'слову', 'нет', 'за', 'тем', 'во', 'например', 'помимо', 'такой', 'будет', 'помилуй', 'тоже', 'скажут',
      'видно', 'пожалуй', 'общеизвестно', 'чье', 'вдобавок', 'впрочем', 'соответственно', 'зачастую', 'сам', 'том', 'там', 'полагается',
      'меня', 'однако', 'слышь', 'итак', 'подобное', 'действительно', 'еще', 'его', 'наше', 'данный', 'твой', 'ай', 'необходимо', 'три',
      'далее', 'тот', 'эм', 'же', 'жаль', 'другими', 'видишь', 'некоторых', 'прежде', 'который', 'допустим', 'положено', 'по-нашему', 'весьма',
      'не', 'ро', 'той', 'этого', 'эти', 'напомним', 'правда', 'выходит', 'перед', 'определенно', 'какая', 'относительно', 'сразу', 'имя',
      'чей', 'ее', 'отом', 'неё', 'это', 'два', 'эту', 'разве', 'вне', 'тобы', 'знамо', 'всем', 'вернее', 'будто', 'вновь', 'которые', 'данным',
      'кроме', 'таким', 'нечто', 'кого', 'больше', 'ом', 'уже', 'прочего', 'очень', 'при', 'те', 'вообще', 'ясно', 'ну', 'октябрь', 'до',
      'на', 'в-третьих', 'ежели', 'общем', 'ещё', 'похоже', 'ибо', 'оно', 'ты', 'сверх', 'значит', 'безусловно', 'правильнее', 'ниего', 'него', 'совсем',
      'вами', 'общем-то', 'всю', 'наш', 'всему', 'всякий', 'ой', 'этой', 'ни', 'со', 'вот', 'день', 'может', 'мб', 'тебя', 'нас',
      'оять', 'хотя', 'прочим', 'этим', 'между', 'ним', 'вероятнее', 'бывало', 'примерно', 'вдруг', 'либо']

## 1.1 Product Dataset

In [5]:
df = pd.read_csv('ProductsDataset.csv')
df.rename(columns = {'descrirption' : 'description'}, inplace = True) 
df = df.fillna('0')
df['description_full'] = df['title'] + ' ' + df['description']
df = df.loc[df['description_full'] != '00']

In [6]:
'''Нормализуем полные описания товаров (название товара + его описание)'''
df['description_full_norm'] = df.apply(lambda x: preprocess_text(x['description_full'], sw), axis=1)

In [ ]:
'''Мини-датасет продукт - таргет 1'''
df_product = pd.DataFrame()
df_product['description'] = df['description_full_norm']
df_product['target'] = 1
df_product.loc[df_product['description'] != 'nan']

## 1.2 Chat Dataset

In [ ]:
'''Отбираем из корпуса со всеми ответами записи, по количеству = количеству продуктовых записей
Это необходимо для сбалансированной выборки для обучения будущей модели'''
prepared_sentences = []
c = 0

with open('prepared_answers.txt', 'r') as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_text(line, sw)
        if spls:
            prepared_sentences.append(spls)
            c += 1
            if c >= len(df_product):
                break

/tmp/ipykernel_802088/3231642506.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [ ]:
'''Мини-датасет вопрос - таргет 0'''
df_chat = pd.DataFrame()
df_chat['description'] = prepared_sentences
df_chat['target'] = 0

In [ ]:
'''Объединяем продуктовый и чат датасеты в один - это наша выборка'''
main_df = pd.concat([df_product, df_chat], ignore_index=True)

## 1.3 ML

In [ ]:
vectorizer = TfidfVectorizer()

X = main_df['description']
y = main_df['target']

X = vectorizer.fit_transform(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [ ]:
log_reg = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

accuracy = (y_pred == y_test).mean()
print(accuracy)

0.9826713176550017


accuracy - 0.98 (!) - это отличный результат. Берем логистическую регрессию в качестве основной модели для нашего классификатора.

In [ ]:
'''Визуализируем работу классификатора'''
vec = vectorizer.transform(['толстовка', 'город'])
log_reg.predict(vec)

array([1, 0])

In [ ]:
'''Сохраняем модель'''
pickle.dump(log_reg, open(f'models/log_reg.sav', 'wb'))

# 2. Chat Branch

In [ ]:
'''Нормализуем ответы из корпуса с вопросами'''
sentences = []
c = 0

with open('answers.txt', 'r') as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_text(line, sw)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

/tmp/ipykernel_4614/4146970320.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [ ]:
'''Обучим модель word2vec на наших вопросах'''
sentences = [i for i in sentences if len(i) > 2]
answer_model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)
pickle.dump(answer_model, open(f'models/answer_model_w2v.sav', 'wb'))

In [ ]:
'''Сложим в индекс все вопросы. Используем библиотеку annoy. 
Проходимся по всем ответам, считаем, что вектор предложения - сумма word2vecов слов (усредненная), которые входят в него'''

answer_index = annoy.AnnoyIndex(100 ,'angular')

answer_index_map = {}
counter = 0

with open('prepared_answers.txt', 'r') as f:
    for line in tqdm_notebook(f):
        try:
            n_w2v = 0
            spls = line.split('\t')
            answer_index_map[counter] = spls[1]
            question = preprocess_text(spls[0], sw)
            vector = np.zeros(100)
            for word in question:
                if word in answer_model.wv:
                    vector += answer_model.wv[word]
                    n_w2v += 1
            if n_w2v > 0:
                vector = vector / n_w2v
            answer_index.add_item(counter, vector)

            counter += 1

            if counter > 500000:
                break
            
        except:
            print(spls)
            pass

answer_index.build(10)
answer_index.save('models/answer.ann')
pickle.dump(answer_index_map, open(f'models/answer_index_map.pkl', 'wb'))

/tmp/ipykernel_4614/2233210317.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


0it [00:00, ?it/s]

True

In [ ]:
'''На входе получаем вопрос от пользователя, который был классифицирован, как другой (не продуктовый)
Выводим тот ответ на вопрос, который ближе всего по смыслу к вопросу пользователя'''
def find_answer(question, answer_model, answer_index, answer_index_map):
    preprocessed_question = preprocess_text(question, sw)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in answer_model.wv:
            vector += answer_model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index_1 = answer_index.get_nns_by_vector(vector, 1)
    return answer_index_map[answer_index_1[0]]

# 3. Product Branch

In [ ]:
df['title_norm'] = df.apply(lambda x: preprocess_text(x['title'], sw), axis=1)

product_list = df['title_norm']
id_list = df['product_id']

product_model = Word2Vec(sentences=product_list, vector_size=100, min_count=1, window=5)
pickle.dump(answer_model, open(f'models/product_model_w2v.sav', 'wb'))

In [ ]:
product_index = annoy.AnnoyIndex(100 ,'angular')

product_index_map = {}
counter = 0

for t, product in enumerate(product_list):
    n_w2v = 0
    product_index_map[counter] = spls[1]
    vector = np.zeros(100)
    for word in product:
        if word in product_model.wv:
            vector += product_model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    product_index.add_item(counter, vector)

    counter += 1
    
product_index.build(10)
product_index.save('models/product.ann')
pickle.dump(product_index_map, open(f'models/product_index_map.pkl', 'wb'))

True

In [ ]:
'''На входе получаем вопрос от пользователя, который был классифицирован, как продуктовый.
Выводим тот id товара, чье название оказалось ближе всего к запросу пользователя'''
def find_product(question, product_model, product_index, product_index_map):
    preprocessed_question = preprocess_text(question, sw)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in product_model.wv:
            vector += product_model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    product_index_1 = product_index.get_nns_by_vector(vector, 1)
    return product_index_map[product_index_1[0]]

# Main algoritm

In [ ]:
def get_answer(question, log_reg, answer_model, answer_index, answer_index_map, product_model, product_index, product_index_map):
    question = preprocess_text(question, sw)
    vec = vectorizer.transform([question])
    clf_result = log_reg.predict(vec).tolist()[0]
    
    if clf_result == 0:
        result = find_answer(question, answer_model, answer_index, answer_index_map)
    elif clf_result == 1:
        result = find_product(question, product_model, product_index, product_index_map)
    return result

In [ ]:
get_answer('Юбка детская ORBY', log_reg, answer_model, answer_index, answer_index_map, product_model, product_index, product_index_map)